<a href="https://colab.research.google.com/github/mdehghani86/AppliedGenAI/blob/main/LangChain_Lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🔗 **LangChain Lab 3: Chains & Agents**  
- **Prof. Dehghani (m.dehghani@northeastern.edu)**  

## **Lab Overview**  
This lab focuses on **automating multi-step reasoning** and **decision-making** in LangChain using **Chains & Agents**.  
You'll learn how to connect multiple components, dynamically execute logic, and use **LLMs to make decisions**.  

---

## **🎯 What You'll Learn in This Lab**  
In this session, you'll explore:  
- 🔹 **Chains in LangChain** → How to connect multiple LLM calls in a sequence.  
- 🔹 **Agents & Tools** → How to create AI-driven agents that reason & act dynamically.  
- 🔹 **Hands-on exercises** → Reinforce concepts with practical coding tasks.  

By the end of this lab, you’ll be able to **build AI workflows** that use **structured pipelines** and **autonomous decision-making** with LangChain! 🚀  


In [ ]:
# ==================================================
# 📌 Installing Required Libraries
# ==================================================
!pip install langchain  # Core framework for working with LLMs
!pip install langchain-community  # Install the community package containing LLMs
!pip install openai==0.28  # OpenAI API package (version 0.28) for GPT models
!pip install langchain-huggingface  # Hugging Face LLM wrapper

In [ ]:
# ==================================================
# 📌 Importing Required Libraries for LangChain Lab
# ==================================================

# ✅ System & Environment Setup
import os  # For setting environment variables, such as API keys

# ✅ Jupyter & Colab Utilities
import ipywidgets as widgets  # For creating interactive input widgets
from IPython.display import clear_output, display  # For managing notebook outputs

# ✅ OpenAI API
import openai  # Direct interaction with OpenAI API (useful for API-based calls)

# ✅ LangChain Components
from langchain.llms import OpenAI  # Wrapper for interacting with OpenAI LLMs
from langchain.chat_models import ChatOpenAI  # For chat-based OpenAI models
from langchain.agents import AgentType, initialize_agent  # For creating AI agents
from langchain.tools import Tool  # For adding external tools to agents
from langchain.memory import ConversationBufferMemory  # For maintaining conversation history
from langchain.prompts import PromptTemplate  # For creating structured prompts

# ✅ Hugging Face Transformers (Only required if using Hugging Face models)
import transformers  # Hugging Face library for pre-trained models

# ✅ Confirmation message
print("✅ All required libraries imported successfully!")


In [ ]:
# ==================================================
# 🔑 OpenAI API Key Setup with Output Clearing
# ==================================================

# ✅ Create input widget for the OpenAI API key
openai_key_input = widgets.Password(
    description="🔑 OpenAI Key:",
    placeholder="Enter your OpenAI API Key",
)

# ✅ Create a button to submit the API key
submit_button = widgets.Button(description="✅ Set API Key")

# ✅ Function to save the OpenAI API key when the button is clicked
def set_api_key(b):
    # Clear previous outputs
    clear_output(wait=True)

    # Display the input field and button again
    display(openai_key_input, submit_button)

    # Retrieve and validate the OpenAI API key
    openai_key = openai_key_input.value.strip()

    # ✅ Set OpenAI API Key
    if openai_key:
        os.environ["OPENAI_API_KEY"] = openai_key
        openai.api_key = openai_key
        print("✅ OpenAI API Key has been set successfully!")
    else:
        print("❌ Please enter a valid OpenAI API Key.")

# ✅ Link button click to the function
submit_button.on_click(set_api_key)

# ✅ Display the input field and button
display(openai_key_input, submit_button)

# 🔗 **Understanding Chains in LangChain**  

One of the **core features** of LangChain is its ability to **create chains**, allowing us to **sequence multiple tasks together**.  
Instead of manually handling each step, **chains automate workflows** by linking components such as **prompts, LLMs, memory, and tools**.  

## ✅ Why Use Chains in LangChain?  
🔹 **Automate multi-step processes** → No need to manually pass outputs between steps.  
🔹 **Create structured AI pipelines** → Chain together **LLMs, retrievers, memory, and tools**.  
🔹 **Enable decision-making AI agents** → Chains help **LLMs interact with external tools** to retrieve and process information dynamically.  

## 🔗 **Using the Pipe (`|`) Operator for Cleaner Chaining**  
LangChain provides a **simplified way** to create chains using the **pipe (`|`) operator**, which allows direct data flow between components.  

### **🔍 Example: Two Ways to Process Input with an LLM**  

🔹 **Without Pipe (`|`)** → Manually format the input, then pass it to the LLM.  
🔹 **With Pipe (`|`)** → Directly chain them together for **automatic execution**.  

Let's compare both approaches in the next code cells! 🚀  


In [ ]:
# ==================================================
# 🔹 **Comparing Two Methods: Manual Execution vs. Pipe (`|`) Operator**
# ==================================================
# This example demonstrates two ways to process an input with an LLM:
#
# 1️⃣ **Without Pipe (`|`)** → Manually format the prompt and pass it to the LLM.
# 2️⃣ **With Pipe (`|`)** → Use LangChain’s `|` operator to create a streamlined sequence.
#
# Using the `|` operator allows for **cleaner, automatic execution**, reducing code complexity.


# ✅ Define a prompt template with a World Cup theme
prompt = PromptTemplate(
    input_variables=["event"],
    template="""
    You are a legendary sports analyst with deep knowledge of World Cup history.
    Fans eagerly await your expert take on the most iconic moments.

    Analyze this legendary World Cup event in max ~20 simple words: {event}
    """
)

# ✅ Initialize the LLM (GPT-4)
llm = ChatOpenAI(model_name="gpt-4", temperature=0.0)

# ==================================================
# ❌ Without Using the Pipe (`|`) - More Manual Steps
# ==================================================
# 1️⃣ Manually format the prompt
formatted_prompt = prompt.format(event="Zidane's 2006 World Cup final red card")

# 2️⃣ Pass it to the LLM manually (Fixed: Removed unnecessary dictionary)
response_without_pipe = llm.invoke(formatted_prompt)

# ✅ Print the response
print("❌ Without Pipe Response:", response_without_pipe.content)

# ==================================================
# ✅ Using the Pipe (`|`) - Cleaner and Automatic Execution
# ==================================================
# 1️⃣ Directly chain the prompt and LLM together
chain = prompt | llm  # This creates a RunnableSequence

# 2️⃣ Run the chain with an input in one step (Fixed: Using correct format)
response_with_pipe = chain.invoke({"event": "Zidane's 2006 World Cup final red card"})

# ✅ Print the response
print("✅ With Pipe Response:", response_with_pipe.content)


### ✋ Hands-On: Single Prompt Chain

In [ ]:
# ==================================================
# ✋ **Hands-On 1: Creating a Single Prompt Chain**
# ==================================================

# 📌 **Task Instructions:**
# 1️⃣ Fill in the missing placeholders (-----) to complete the code.
# 2️⃣ Ensure the Prompt Template correctly replaces {topic}.
# 3️⃣ Run the code and verify GPT-4 generates a response.

# ✅ Step 1: Import necessary modules
from langchain.prompts import -----  # Import the correct class
from langchain.chat_models import -----  # Import the correct class

# ✅ Step 2: Define a Prompt Template
prompt_template = -----(
    input_variables=["-----"],  # Placeholder for dynamic input
    template="Explain {topic} in simple terms using no more than 15 words."
)

# ✅ Step 3: Initialize GPT-4 model
llm_ChatGPT = -----(model_name="gpt-4")  # Load GPT-4 model

# ✅ Step 4: Create a runnable chain using `|` (pipe operator)
chain = prompt_template ----- llm_ChatGPT  # Use the correct operator to chain them

# ✅ Step 5: Run the chain with a sample input
response = chain.invoke({"topic": "Collective Intelligence"})

# ✅ Step 6: Display results
print("🔹 **Generated Prompt:**", prompt_template.format(topic="Collective Intelligence"))
print("🔹 **LLM Response:**", response.-----)  # Extract and display response content


# 🤖 Multi-LLM Pipelines with Chaining  

## 🔹 What is Multi-LLM Chaining?  
Multi-LLM chaining is a method where **multiple AI models** collaborate in a **step-by-step sequence** to handle complex tasks efficiently. Instead of a single model doing everything, **each AI is specialized** for a specific function, ensuring **better accuracy, efficiency, and interpretability**.  

---

## ✅ **Key Benefits of Multi-LLM Chaining**
✔️ **Task Specialization** – Each model is optimized for a **specific role**, leading to **higher accuracy**.  
✔️ **Improved Efficiency** – Models **focus on one task at a time**, reducing processing load and response time.  
✔️ **Scalability** – Easily extendable by adding **more AI models** for deeper analysis.  
✔️ **Transparency & Interpretability** – Step-by-step outputs **show AI reasoning**, making results easier to trust.  
✔️ **Error Reduction** – If an early step **detects errors**, later models can **correct them** for a refined output.  

---

## 🚀 Real-World Applications of Multi-LLM Chaining
🔹 **Legal AI:** One model extracts case details, another predicts legal outcomes.  
🔹 **Customer Support:** One model classifies sentiment, another generates a response.  
🔹 **Content Writing:** One model creates content, another summarizes or edits it.  
🔹 **Fake News Detection:** One model extracts claims, another fact-checks them.  

By chaining **specialized AI models**, we can **enhance AI reasoning**, improve accuracy, and build more **intelligent, structured workflows** across different industries. 🌟  

---

## 🏥 Example: Medical Diagnosis Chain  
In a medical AI pipeline:  
1️⃣ **GPT-4 Turbo** → Analyzes symptoms and suggests possible conditions.  
2️⃣ **GPT-4-01** → Evaluates the list and selects the **most likely condition** with reasoning.  

This structured approach **breaks down decision-making into logical steps**, leading to **more reliable outputs**.


In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

# ================================
# ✅ Step 1: Define Two LLMs
# ================================

# 🔹 First LLM (GPT-4 Turbo) - Medical AI: Finds Possible Conditions
llm_medical = ChatOpenAI(model_name="gpt-4-turbo", temperature=0.0)

# 🔹 Second LLM (GPT-4-01) - Reasoning AI: Selects Best Condition & Justifies
llm_reasoning = ChatOpenAI(model_name="gpt-4-1106-preview", temperature=0.0)

# ================================
# ✅ Step 2: Create Prompt Templates
# ================================

# 🔹 Prompt for Medical AI (Find Possible Conditions)
prompt_medical = PromptTemplate(
    input_variables=["symptoms"],
    template="""
    You are an AI medical assistant. Based on the symptoms provided, suggest up to 3 possible conditions.

    Symptoms: {symptoms}

    Respond in a **comma-separated list** (e.g., "Flu, COVID-19, Pneumonia").
    """
)

# 🔹 Prompt for Reasoning AI (Pick the Most Likely Condition)
prompt_reasoning = PromptTemplate(
    input_variables=["conditions"],
    template="""
    You are an AI doctor. Based on the possible conditions listed, pick the **most probable** one and provide a **brief reason**.

    Possible conditions: {conditions}

    Respond with **only the best condition + reasoning in ≤15 words**.
    """
)

# ================================
# ✅ Step 3: Create the Chain (Using `|` Operator)
# ================================

# 🔹 Chain Explanation:
# 1️⃣ GPT-4 Turbo gets symptoms → outputs possible conditions
# 2️⃣ The conditions are passed automatically to GPT-4-01
# 3️⃣ GPT-4-01 picks **one most likely condition** & gives a short reason

medical_chain = (
    prompt_medical  # Step 1: Format symptoms into a medical prompt
    | llm_medical   # Step 2: Use GPT-4 Turbo to find possible conditions
    | (lambda x: {"conditions": x.content})  # Step 3: Extract conditions as input for next LLM
    | prompt_reasoning  # Step 4: Format the conditions for reasoning AI
    | llm_reasoning  # Step 5: Use GPT-4-01 to pick the best condition
)

In [ ]:
# 🔹 Test Case 1: Common Cold Symptoms
symptoms_input = "runny nose, sneezing, mild headache"

response = medical_chain.invoke({"symptoms": symptoms_input})

print("🔍 Possible Conditions (GPT-4 Turbo):", response.content)


In [ ]:
# 🔹 Test Case 2: Severe Flu-like Symptoms
symptoms_input = "fever, chills, muscle pain, fatigue"

response = medical_chain.invoke({"symptoms": symptoms_input})

print("🔍 Possible Conditions (GPT-4 Turbo):", response.content)


In [ ]:
# 🔹 Test Case 3: Stomach-related Symptoms
symptoms_input = "nausea, vomiting, diarrhea, stomach cramps"

response = medical_chain.invoke({"symptoms": symptoms_input})

print("🔍 Possible Conditions (GPT-4 Turbo):", response.content)


## ✋ Hands-On Exercise: Customer Review Analysis & Automated Response

In [ ]:
# ==================================================
# ✋ **Hands-On: Customer Review Sentiment & AI Response**
# ==================================================
# 📌 **Task Instructions:**
# 1️⃣ Fill in the missing placeholders (`-----`) to complete the code.
# 2️⃣ Ensure both Prompt Templates correctly embed {review} and {sentiment}.
# 3️⃣ Run the code and verify the AI-generated sentiment & response.

# ✅ Step 1: Import necessary modules

# ✅ Step 2: Initialize two different LLMs
llm_sentiment = -----("gpt-4-turbo")  # GPT-4 Turbo for sentiment analysis
llm_response = -----("gpt-4-1106-preview")  # GPT-4-01 for response generation

# ✅ Step 3: Define the first Prompt Template (Sentiment Analysis)
sentiment_prompt = -----(
    input_variables=["-----"],  # Define the input variable for review input
    template="""
    You are an AI assistant analyzing customer sentiment.

    Review: {review}

    Respond with either "Positive", "Neutral", or "Negative".
    """
)

# ✅ Step 4: Define the second Prompt Template (Automated Response)
response_prompt = -----(
    input_variables=["-----"],  # Define the input variable for sentiment classification
    template="""
    You are an AI customer support agent. Based on the sentiment, generate a short, polite response.

    Sentiment: {sentiment}

    Response (≤ 15 words):
    """
)

# ✅ Step 5: Create Runnable Chains using the `|` operator
sentiment_chain = ----- | -----  # Chain the sentiment prompt with GPT-4 Turbo
response_chain = ----- | -----  # Chain the response prompt with GPT-4-01

# ✅ Step 6: Run the pipeline

# Step 6.1: Sample customer review input
customer_review = "The product is amazing! Great quality and fast delivery. Will buy again."

# Step 6.2: Analyze sentiment using GPT-4 Turbo
sentiment_result = sentiment_chain.-----({"review": customer_review})  # Call the function to invoke the model

# Step 6.3: Generate AI response using GPT-4-01
response_text = response_chain.-----({"sentiment": sentiment_result.-----})  # Ensure correct content extraction

# ✅ Step 7: Display results
print("🔍 **Detected Sentiment:**", sentiment_result.-----)  # Extract response content
print("\n💬 **AI Response:**", response_text.-----)  # Extract AI-generated response


#+++++++ Example Output +++++++#
#🔍 **Detected Sentiment:** Positive
#💬 **AI Response:** Thank you for your kind words! We’re glad you loved it! 😊


## ✋ Hands-On Exercise: Customer Review Analysis & Automated Response (Merged Chain)

In [ ]:
# ==================================================
# ✋ **Hands-On: Merged Chain for Customer Review Sentiment & AI Response**
# ==================================================
# 📌 **Task Instructions:**
# 1️⃣ Fill in the missing placeholders (`-----`) to complete the chain.
# 2️⃣ Ensure sentiment analysis flows correctly into the response generation.
# 3️⃣ Run the code and verify AI-generated responses.

# ✅ The `PromptTemplate` and LLMs (`llm_sentiment` and `llm_response`) are already defined.

# ✅ Step 5: Create a Single Runnable Chain (Merged Approach)
full_chain = (
    -----  # Step 1: Start with the sentiment prompt
    | -----  # Step 2: Pass it to the sentiment analysis LLM
    | (lambda x: {"sentiment": x.content})  # Step 3: Extract sentiment result
    | -----  # Step 4: Format sentiment into response prompt
    | -----  # Step 5: Pass it to the response generation LLM
)

# ✅ Step 6: Run the pipeline

# Sample customer review input
customer_review = "The product is amazing! Great quality and fast delivery. Will buy again."

# Run the full pipeline in one step
response = full_chain.invoke({"review": customer_review})

# ✅ Step 7: Display results
print("💬 **AI Response:**", response.content)  # Extract AI-generated response


# 🤖 AI Agents: Beyond Basic LLM Chaining  

## 🔹 What are AI Agents?  
Unlike simple LLM pipelines where models work in a **fixed sequence**, **AI agents** are **more flexible** and can **make decisions dynamically**. They can interact with **external tools, APIs, memory, and reasoning frameworks** to **adapt their responses** based on the situation.

---

## 🔍 **LLM Chaining vs. AI Agents**
| Feature           | LLM Chaining                     | AI Agents                      |
|------------------|--------------------------------|--------------------------------|
| **Execution**    | Fixed sequence of steps        | Dynamic, adaptive behavior    |
| **Decision-Making** | Follows predefined logic       | Can reason and choose actions |
| **Interactivity** | Limited to internal logic      | Can use APIs, databases, tools |
| **Memory**       | No long-term state             | Can remember previous actions |

---

## ✅ **Why Use AI Agents?**
✔️ **Decision-Making** – Agents **select actions** dynamically instead of just following a script.  
✔️ **Tool Integration** – Can access **APIs, databases, and external tools** like search engines or calculators.  
✔️ **Memory** – Stores past interactions to **improve responses over time**.  
✔️ **Multi-Step Reasoning** – Can **plan**, execute, and refine responses based on feedback.  

---

## 🚀 Real-World Applications of AI Agents
🔹 **Customer Support AI:** Detects user needs, queries databases, and provides **real-time support**.  
🔹 **Financial AI:** Analyzes market trends, retrieves **live stock prices**, and recommends investments.  
🔹 **Research Assistants:** Searches the web, **extracts insights**, and summarizes articles.  
🔹 **Automated Workflow Agents:** Interact with **multiple APIs** to execute complex business tasks.  

---

## ⚡ Next Steps: Building an AI Agent  
Now, let’s **create an AI agent** that can **reason, interact with tools, and make decisions** dynamically! 🚀  


# 🧮 Lab: Building a Math Solver Agent  

In this lab, we create an **AI agent** that can solve **math problems dynamically** using a **calculator tool**. The agent **decides** whether to perform calculations itself or use external tools, showcasing **reasoning and tool integration** in LangChain.  

The agent is executed using **`.run()`**, which allows it to process user queries and decide on actions dynamically.  

---

## 🔹 What is a Tool in LangChain?  
A **tool** in LangChain is an **external function** that an agent can call to perform **specialized tasks** beyond just text generation. Tools help **extend AI capabilities**, allowing it to:  
✔️ **Perform calculations** (e.g., a calculator tool)  
✔️ **Query APIs** (e.g., fetch stock prices, weather updates)  
✔️ **Search databases** (e.g., retrieve company records)  

In this lab, we use a **calculator tool** to enable precise **math computation**, ensuring accurate results instead of relying solely on an LLM’s reasoning.  

In [ ]:
# ================================
# 🏗️ Build the Math Solver Agent
# ================================
# ✅ This cell initializes the agent with a reasoning LLM and a calculator tool.

from langchain.chat_models import ChatOpenAI  # This imports the OpenAI-powered chatbot model for conversational AI
from langchain.agents import initialize_agent, AgentType  # This is used to initialize an agent with a specific type
from langchain.tools import Tool  # Tool class is used to define custom tools that the agent can use
import operator  # Importing the operator module to perform mathematical and logical operations

# ✅ Step 1: Define the LLM
llm = ChatOpenAI(model_name="gpt-4-turbo", temperature=0.0)

import operator  # Importing the operator module for mathematical operations

# ✅ Step 2: Create a Calculator Tool
def calculator_tool(expression: str) -> str:
    """
    Securely evaluates a mathematical expression and returns the computed result.

    This function allows basic arithmetic operations while restricting access to
    potentially dangerous built-in functions.

    Supported operations:Addition (+), Subtraction (-), Multiplication (*)
    - Division (/), Exponentiation (**), Floor Division (//), Modulus (%)

    Parameters:
    expression (str): A valid mathematical expression in string format
                      (e.g., "5 + 3", "10 * 2", "8 ** 2").

    Returns:
    str: The computed result as a string, or an error message if the input is invalid.
    """
    try:
        # ✅ Securely evaluate the mathematical expression
        # - `eval()` computes the arithmetic operation safely.
        # - `{"__builtins__": {}}` removes access to all built-in functions, preventing security risks.
        # - `operator.__dict__` limits the allowed operations to those defined in the operator module.
        result = eval(expression, {"__builtins__": {}}, operator.__dict__)

        # ✅ Convert the result to a string before returning it
        return str(result)

    except Exception as e:
        # ✅ Handle errors gracefully, such as:
        # - Invalid mathematical expressions (e.g., "five plus three" instead of "5 + 3")
        # - Unsupported operations
        # - Division by zero
        return f"Error: {str(e)}"

calculator = Tool(
    name="Calculator",
    func=calculator_tool,
    description="Use this tool to perform basic arithmetic calculations."
)

# ✅ Step 3: Initialize the Math Solver Agent
math_solver_agent = initialize_agent(
    tools=[calculator],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)


# 🔹 Why Do We Need an LLM for the Math Solver Agent?  

The **LLM (GPT-4 Turbo)** is essential for enabling **natural language understanding and reasoning** in the **Math Solver Agent**.  

### **✅ Why Keep the LLM?**  
✔ Interprets **human-like queries** (e.g., `"Multiply 10 by 3"` → `"10 * 3"`)  
✔ Decides **when to use the calculator tool**  
✔ Provides **error handling and explanations** in natural language  
✔ Responds to **unsupported queries** instead of failing  

### **❌ What Happens Without It?**  
🔹 Only strict expressions (e.g., `"5 + 3"`) work, no **language understanding**  
🔹 The tool **cannot reason** or decide how to handle a query  
🔹 The agent **becomes a basic calculator**, losing flexibility  

🚀Using the LLM makes the agent **more intelligent, flexible, and user-friendly** beyond just executing math operations.  


In [ ]:
# ================================
# 🧪 Test the Math Solver Agent
# ================================
# ✅ This cell runs the agent with different math problems.

# 🔹 Test Cases (Increasing Difficulty):
test_cases = [
    "What is 42 * (8 + 3)?",  # Simple arithmetic
    "Solve for x: 3x + 5 = 20.",  # Equation solving
    "Integrate (3x^2 + 2x - 5) dx.",  # Advanced calculus (integration)
]

# 🔹 Run the agent on each test case one by one
for query in test_cases:
    response = math_solver_agent.run(query)
    print(f"🧮 Query: {query}")
    print(f"📢 Response: {response}\n")


## 💡 Did You Realize? Your Agent is Smarter Than You Think!

### ❓ Why does the Math Solver Agent understand `"Multiply 10 by 3"` even though there is no prompt?  
✅ The agent uses **`ZERO_SHOT_REACT_DESCRIPTION`**, which **automatically prompts the LLM** behind the scenes.  

### ❓ How does the LLM convert `"Multiply 10 by 3"` into `"10 * 3"`?  
✅ LangChain **asks the LLM to interpret the query** and map words to mathematical symbols before calling the calculator tool.  

### ❓ What happens if we remove the LLM?  
✅ The agent **won’t understand** `"Multiply 10 by 3"`, and only exact expressions like `"10 * 3"` will work.  

### ❓ Does the definition of a tool get passed to the LLM?  
✅ Yes! The agent **passes the tool’s description** to the LLM so it knows when and how to use it.  

### ❓ Can an agent have multiple tools?  
✅ Yes! An agent can use **multiple tools**, and the LLM will decide which one to call based on the query.

In [ ]:
# ================================
# ✋ Hands-On: Build a Unit Conversion Agent
# ================================
# 📌 **Task Instructions:**
# 1️⃣ Fill in the missing placeholders (`-----`) to complete the code.
# 2️⃣ Ensure the tool and agent are correctly initialized.
# 3️⃣ Test the agent by providing a conversion query.

from langchain.chat_models import ChatOpenAI  # Importing OpenAI-powered LLM
from langchain.agents import initialize_agent, AgentType  # For agent initialization
from langchain.tools import Tool  # Tool class to define custom tools for the agent

# ================================
# ✅ Step 1: Define the LLM
# ================================
# 🔹 Fill in the placeholder to define the LLM
llm = -----("gpt-4-turbo", temperature=0.0)

# ================================
# ✅ Step 2: Create a Unit Conversion Tool
# ================================
def unit_conversion_tool(query: str) -> str:
    """
    Handles unit conversions for basic metrics like length, weight, and temperature.

    Supported conversions:
    - Miles to Kilometers
    - Kilograms to Pounds
    - Celsius to Fahrenheit

    The function extracts the numerical value from the query string, performs the
    specified conversion, and returns the result as a formatted string.

    Returns:
    str: The conversion result or an error message if the query is unsupported or invalid.
    """
    try:
        if "miles to kilometers" in query.lower():
            miles = float(query.split()[0])
            return f"{miles} miles is {miles * 1.60934:.2f} kilometers."
        elif "kilograms to pounds" in query.lower():
            kg = float(query.split()[0])
            return f"{kg} kilograms is {kg * 2.20462:.2f} pounds."
        elif "celsius to fahrenheit" in query.lower():
            celsius = float(query.split()[0])
            return f"{celsius}°C is {celsius * 9/5 + 32:.2f}°F."
        else:
            return (
                "Unsupported conversion. Supported conversions:\n"
                "- Miles to Kilometers\n"
                "- Kilograms to Pounds\n"
                "- Celsius to Fahrenheit"
            )
    except Exception as e:
        return f"Error: Unable to process the query. Details: {str(e)}"


# 🔹 Fill in the placeholder to define the tool
unit_converter = Tool(
    name="Unit Converter",
    func=-----,  # Function to handle unit conversions
    description="Use this tool to convert units like miles to kilometers, kilograms to pounds, and Celsius to Fahrenheit."
)

# ================================
# ✅ Step 3: Initialize the Unit Conversion Agent
# ================================
# 🔹 Fill in the placeholder to initialize the agent
unit_conversion_agent = initialize_agent(
    tools=[-----],  # Tool for unit conversion
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# ================================
# ✅ Step 4: Test the Unit Conversion Agent
# ================================
# 🔹 Example query for conversion
query = "5 miles to kilometers"

# 🔹 Run the agent with the query
response = unit_conversion_agent.run(query)

# 🔹 Print the response
print("🔄 Unit Conversion Agent Response:\n", response)


# 📈 Stock Market Agent with Simplified Analytics

This lab demonstrates how to create a **Stock Market Agent** that retrieves and analyzes AAPL stock data. The agent calculates the **latest close price**, **price changes**, and determines the **trend direction** (upwards or downwards) using historical data. By leveraging LangChain tools and an LLM, the agent provides concise and actionable insights into stock performance.


In [ ]:
# ================================
# 📈 Stock Market Agent with Simplified Analytics
# ================================
# ✅ This agent retrieves the latest AAPL stock data and provides concise analytics.

import pandas as pd  # For handling CSV data
from langchain.chat_models import ChatOpenAI  # LLM for reasoning and insights
from langchain.agents import initialize_agent, AgentType  # Agent framework
from langchain.tools import Tool  # Tool integration

# ================================
# ✅ Step 1: Load AAPL Stock Data from Dropbox
# ================================
# 🔹 Load the historical stock data
csv_url = "https://www.dropbox.com/scl/fi/ysqxvj39gx2bkl4husg3y/HistoricalData_1739205046441.csv?rlkey=36q4rjpvvmwt9fyf3fftxwvb3&dl=1"
stock_data = pd.read_csv(csv_url)

# 🔹 Format the "Close/Last" column (remove "$" sign and convert to float)
stock_data['Close/Last'] = stock_data['Close/Last'].str.replace('$', '').astype(float)

# ================================
# ✅ Step 2: Define the Stock Analytics Tool
# ================================
def get_aapl_analytics(query: str) -> str:
    """
    Fetches the latest stock data and provides simplified analytics:
    - Latest close price
    - Percentage price change compared to the previous day
    - Trend direction (upward/downward)
    """
    # Get the latest and previous stock data
    latest_entry = stock_data.iloc[-1]  # Most recent row
    previous_entry = stock_data.iloc[-2]  # Second last row

    # Calculate price change and percentage change
    price_change = latest_entry['Close/Last'] - previous_entry['Close/Last']
    price_change_percent = (price_change / previous_entry['Close/Last']) * 100

    # Determine the trend direction
    trend = "upwards 📈" if price_change > 0 else "downwards 📉"

    # Create a simplified response
    response = (
        f"📅 Date: {latest_entry['Date']}\n"
        f"🔹 Close Price: ${latest_entry['Close/Last']}\n"
        f"🔹 Price Change: {price_change:.2f} ({price_change_percent:.2f}%)\n"
        f"🔹 Trend: The stock is trending {trend}.\n"
    )
    return response

# 🔹 Register the analytics tool
analytics_tool = Tool(
    name="AAPL Stock Analytics",
    func=get_aapl_analytics,
    description="Provides analytics for AAPL stock, including close price, price changes, and trend direction."
)

# ================================
# ✅ Step 3: Initialize the Stock Market Agent
# ================================
# 🔹 Initialize the GPT-4 Turbo model for reasoning
llm = ChatOpenAI(model_name="gpt-4-turbo", temperature=0.0)

# 🔹 Setup the agent with the analytics tool
stock_agent = initialize_agent(
    tools=[analytics_tool],  # The agent uses the analytics tool to fetch and analyze data
    llm=llm,  # LLM for reasoning and generating insights
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,  # A simple reactive agent
    verbose=True  # Display detailed logs for better understanding
)

# ================================
# ✅ Step 4: Run the Stock Market Agent
# ================================
# 🔹 Example query to analyze AAPL stock
query = "Provide the latest stock data for AAPL."

# 🔹 Get the agent's response
response = stock_agent.run(query)

# 🔹 Print the response
print("📊 Stock Market Agent Response:\n", response)


In [ ]:
# ================================
# 📈 Chaining Analytics and Graphing Agents
# ================================
# ✅ This cell demonstrates chaining two agents: one for analytics and another for graphing.

import matplotlib.pyplot as plt  # For graphing stock price trends
from langchain.tools import Tool

# ================================
# ✅ Step 1: Define the Graphing Tool
# ================================
def generate_stock_graph(query: str) -> str:
    """
    Generates a stock price trend graph for AAPL over the last 10 days.
    """
    # Extract the last 10 days of data
    recent_data = stock_data.tail(10)
    dates = recent_data['Date']
    close_prices = recent_data['Close/Last']

    # Create the graph
    plt.figure(figsize=(10, 6))
    plt.plot(dates, close_prices, marker='o', linestyle='-', color='blue')
    plt.title("AAPL Stock Price Trend (Last 10 Days)", fontsize=14)
    plt.xlabel("Date", fontsize=12)
    plt.ylabel("Close Price ($)", fontsize=12)
    plt.xticks(rotation=45, fontsize=10)
    plt.grid(True)
    plt.tight_layout()
    plt.savefig("aapl_stock_trend.png")  # Save the graph as an image
    plt.close()

    return "📊 Stock trend graph generated successfully: 'aapl_stock_trend.png'"

# 🔹 Register the graphing tool
graph_tool = Tool(
    name="Graph Generator",
    func=generate_stock_graph,
    description="Generates a graph of AAPL's stock price trend over the last 10 days."
)

# ================================
# ✅ Step 2: Chain the Agents
# ================================

# 🔹 Analytics Agent (from previous step)
analytics_agent = stock_agent  # This uses the existing analytics agent

# 🔹 Graphing Agent
llm_graph = ChatOpenAI(model_name="gpt-4-turbo", temperature=0.0)
graphing_agent = initialize_agent(
    tools=[graph_tool],  # The graphing tool generates the stock graph
    llm=llm_graph,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# 🔹 Chain Execution: Run analytics first, then graphing
query = "Provide the latest stock data for AAPL."

# Step 1: Run the analytics agent
analytics_response = analytics_agent.run(query)
print("📈 Analytics Agent Response:\n", analytics_response)

# Step 2: Run the graphing agent
graph_response = graphing_agent.run("Generate a graph for AAPL stock trend.")
print("\n📊 Graphing Agent Response:\n", graph_response)


In [ ]:
# ================================
# 📈 Stock Market and Investment Advice System
# ================================
# ✅ This system fetches stock analytics and provides investment advice by chaining two agents.

import pandas as pd  # For handling CSV data
from langchain.chat_models import ChatOpenAI  # LLM for reasoning and advice
from langchain.agents import initialize_agent, AgentType  # Agent framework
from langchain.tools import Tool  # Tool integration

# ================================
# ✅ Step 1: Load AAPL Stock Data from Dropbox
# ================================
csv_url = "https://www.dropbox.com/scl/fi/ysqxvj39gx2bkl4husg3y/HistoricalData_1739205046441.csv?rlkey=36q4rjpvvmwt9fyf3fftxwvb3&dl=1"
stock_data = pd.read_csv(csv_url)

# 🔹 Format the "Close/Last" column (remove "$" sign and convert to float)
stock_data['Close/Last'] = stock_data['Close/Last'].str.replace('$', '').astype(float)

# ================================
# ✅ Step 2: Define the Stock Analytics Tool
# ================================
def get_aapl_analytics(query: str) -> str:
    """Fetch the latest stock data and provide analytics."""
    # Get the latest and previous stock data
    latest_entry = stock_data.iloc[-1]
    previous_entry = stock_data.iloc[-2]

    # Calculate price change and percentage change
    price_change = latest_entry['Close/Last'] - previous_entry['Close/Last']
    price_change_percent = (price_change / previous_entry['Close/Last']) * 100

    # Determine the trend direction
    trend = "upwards 📈" if price_change > 0 else "downwards 📉"

    # Generate analytics summary
    analytics_summary = (
        f"📅 Date: {latest_entry['Date']}\n"
        f"🔹 Close Price: ${latest_entry['Close/Last']}\n"
        f"🔹 Price Change: {price_change:.2f} ({price_change_percent:.2f}%)\n"
        f"🔹 Trend: The stock is trending {trend}.\n"
    )
    return analytics_summary

# 🔹 Register the analytics tool
analytics_tool = Tool(
    name="AAPL Stock Analytics",
    func=get_aapl_analytics,
    description="Provides analytics for AAPL stock, including close price, price changes, and trend direction."
)

# ================================
# ✅ Step 3: Define the Investment Advice Agent
# ================================
def get_investment_advice(analytics_summary: str) -> str:
    """
    Provides a brief investment recommendation based on the stock analytics.
    """
    advice_prompt = (
        f"The following is the latest stock analysis for AAPL:\n"
        f"{analytics_summary}\n\n"
        f"As a financial advisor, provide brief investment advice (e.g., buy, sell, or hold) "
        f"with reasoning in 2 sentences."
    )
    return llm_investment.predict(advice_prompt)

# 🔹 Register the investment advice tool
investment_tool = Tool(
    name="Investment Advice Generator",
    func=get_investment_advice,
    description="Generates investment advice based on the latest stock analytics."
)

# ================================
# ✅ Step 4: Initialize the Two-Agent System
# ================================

# 🔹 Define the LLMs
llm_analytics = ChatOpenAI(model_name="gpt-4-turbo", temperature=0.0)  # For stock analysis
llm_investment = ChatOpenAI(model_name="gpt-4-turbo", temperature=0.0)  # For investment advice

# 🔹 Create the stock analytics agent
stock_agent = initialize_agent(
    tools=[analytics_tool],
    llm=llm_analytics,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# 🔹 Create the investment advice agent
investment_agent = initialize_agent(
    tools=[investment_tool],
    llm=llm_investment,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# ================================
# ✅ Step 5: Chain the Two Agents
# ================================
# 🔹 Get stock analytics from the first agent
stock_query = "Provide the latest stock analytics for AAPL."
analytics_output = stock_agent.run(stock_query)

# 🔹 Pass the analytics to the second agent for investment advice
investment_query = analytics_output
investment_advice = investment_agent.run(investment_query)

# ================================
# ✅ Step 6: Display the Final Output
# ================================
print("📊 Stock Analytics:\n", analytics_output)
print("\n💡 Investment Advice:\n", investment_advice)
